In [ ]:
!pip install tensorboard
!pip install torchinfo
!pip install --upgrade torch torchvision

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import time
import datetime
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from fastprogress import master_bar, progress_bar
from torch.utils.tensorboard import SummaryWriter
import torchinfo
import numpy as np
import random

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader

from sklearn.metrics import accuracy_score, confusion_matrix

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def to_oneHot(label, numberOfClass):
  oneHot_label = torch.zeros(label.shape[0],numberOfClass).to(device)
  for i in range(label.shape[0]):
    oneHot_label[i][label[i]]=1
  return oneHot_label

def train_step(mlp_model, criterion, optim, input,tabulars, label, batch_size, numberOfClass):
    mlp_model.train()
    img = img.to(device)
    label = label.to(device)
    label = to_oneHot(label,numberOfClass)
    input = img.view(batch_size, -1)
    step_loss = train_step(optim, criterion, mlp_model, input, tabulars, label)
    optim.zero_grad()
    output = mlp_model(input, tabulars)
    loss = criterion(output, label)
    loss.backward()
    optim.step()
    step_loss = loss.item()
    return step_loss

def train_epoch(mlp_model, loader, criterion, optim,mb,numberOfClass):
    epoch_loss_accumulated = 0.0
    for img, tabulars, label in  progress_bar(loader,parent = mb):
      batch_size = img.size(0)
      epoch_loss_accumulated += train_step(mlp_model,criterion,optim, img, tabulars, label, batch_size,numberOfClass)
    return epoch_loss_accumulated/len(loader)

In [ ]:
def validation_epoch(mlp_model, val_loader, criterion, classes = None):
    mlp_model.eval()
    epoch_loss = 0.0
    all_labels = []
    all_predictions = []
    
    with torch.no_grad():
      for images, tabulars, labels in val_loader:
        all_labels.extend(labels.numpy())  
        labels = labels.to(device)
        labels = to_oneHot(label,numberOfClass)
        predictions = mlp_model(images.to(device), tabulars)
        all_predictions.extend(torch.argmax(predictions, dim=1).cpu().numpy())

        loss = criterion(predictions, labels)

        epoch_loss += loss.item()    

    return epoch_loss / len(val_loader), accuracy_score(all_labels, all_predictions) * 100

In [ ]:
def train_model(mlp_model, train_loader, valid_loader, criterion, optim, number_epochs,numberOfClass,scheduler=None):
  train_history = []
  valid_history = []
  accuracy_history = []
  now = datetime.datetime.now()
  date_time = now.strftime("%m%d%Y_%H%M%S")
  name = 'runs/'+mlp_model.name+'_'+date_time
  tensorBoard_writer = SummaryWriter(name) 
  mb = master_bar(range(1, number_epochs+1))
  for epoch in mb:
      start_time = time.time()     
      train_loss = train_epoch(mlp_model, train_loader, criterion, optim,mb,numberOfClass)
      train_history.append(train_loss)
      print("Training epoch {} | Loss {:.6f} | Time {:.2f} seconds"
            .format(epoch + 1, train_loss, time.time() - start_time))
      
      start_time = time.time()
      val_loss, acc = validation_epoch(mlp_model, valid_loader, criterion)
      valid_history.append(val_loss)
      accuracy_history.append(acc)
      print("Validation epoch {} | Loss {:.6f} | Accuracy {:.2f}% | Time {:.2f} seconds"
            .format(epoch + 1, valid_loss, acc, time.time() - start_time))
      if(scheduler):
        scheduler.step() 
      # Se carga en tensorBoard #Loss #Validation en train y val
      tensorBoard_writer.add_scalar(tag="Train Loss", scalar_value=train_loss, global_step=epoch)
      tensorBoard_writer.add_scalar(tag="Validation Loss", scalar_value=val_loss, global_step=epoch)
      tensorBoard_writer.add_scalar(tag="Validation Accuracy", scalar_value=acc, global_step=epoch)
  tensorBoard_writer.close()

In [ ]:
# ACA NECESITO ARMAR Vector de vectores img, tabulares, label -------------------------------------------------------------------------------------------------------------------------------------
def get_dataloaders(train_transf,batch_size):
  train_dataset = ImageFolder("train_set",transform=train_transf)
  train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True,transform=transforms.ToTensor())
# ACA NECESITO ARMAR Vector de vectores img, tabulares, label -------------------------------------------------------------------------------------------------------------------------------------

  BATCH_SIZE = batch_size

  # Separamos en train y validation
  train_size = int(0.8 * len(train_dataset))
  valid_size = len(train_dataset) - train_size

  train, validation = torch.utils.data.random_split(train_dataset, [train_size,valid_size])

  print(f"{len(train)} Training images, {len(validation)} Validation images")

  # Podemos usar data loaders como vimos en el práctico.
  train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True,pin_memory=True)
  valid_loader = DataLoader(validation, batch_size=BATCH_SIZE, pin_memory=True)
  
  return train_loader, valid_loader, len(train_dataset.classes)

In [ ]:
BATCH_SIZE = 32

# Creamos los dataloaders
train_transform = transforms.Compose([
    transforms.Resize([256,192]),
    transforms.ToTensor()
])

# Creamos el loaders
train_loader, valid_loader, num_classes = get_dataloaders(transform_iterator,BATCH_SIZE)

In [ ]:
%load_ext tensorboard

In [ ]:
class MLP_Model(nn.Module):
  def __init__(self,name="MLP_MODEL", num_classes):
    super().__init__()
    self.name = name
    self.conv1 = nn.Conv2d(3, 64, 4, stride=2, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.conv2 = nn.Conv2d(64, 128, 4, stride=2, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(128)
    self.conv3 = nn.Conv2d(128, 256, 4, stride=2, padding=1, bias=False)
    self.bn3 = nn.BatchNorm2d(256)
    self.conv4 = nn.Conv2d(256, 64, 4, stride=2, padding=1, bias=False)
    self.bn4 = nn.BatchNorm2d(64)
    self.linear1 = nn.Linear(64*16*12+6, 1024)
    self.linear2 = nn.Linear(1024, 512)
    self.linear3 = nn.Linear(512, 128)
    self.linear4 = nn.Linear(128, 64)
    self.out = nn.Linear(64, num_classes)


  def forward(self, x,tabulars):
    # entrada de 256*192
    x = x.view(x.size(0), 1, 28, 28)
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.relu(self.bn4(self.conv4(x)))
    x = x.view(x.size(0), -1)
    x = torch.concat([x,tabulars],-1)
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = F.relu(self.linear4(x))
    x = F.sigmoid(self.out(x))
    return x

In [ ]:
torchinfo.summary(MLP_Model())

In [ ]:
# Definimos el modelo y el optimizador
BATCH_SIZE = 32
epochs = 50
LR = 2e-4
B = [0.5,0.999]
mlp_model = MLP_Model("MLP MODEL", num_classes).to(device)
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=LR,betas=B)
crit = nn.BCELoss()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3,5,8,10],gamma=0.1, verbose=True)


In [ ]:
train_model(mlp_model, train_loader, valid_loader, crit, optimizer, epochs, num_classes, scheduler)

In [ ]:
# Guardado del modelo

torch.save(mlp_model.state_dict(),mlp_model.name+".dat")

In [ ]:
%tensorboard --logdir=runs/